In [3]:
import os 
os.getcwd()
os.chdir('C://kiosks_')

In [15]:
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
import json


openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [9]:
with open('main\\components\\menu_maneger\\pdf_dir\\reviewed_menu_data.json', 'r') as menu_json_file:
    data = json.load(menu_json_file)
    data

In [11]:
data

[{'item_name': 'Dhaniya Paneer Tikka',
  'price': '675',
  'description': 'Apricot, almond slivers, corn chat, pickled mango dip'},
 {'item_name': 'Chicken or Paneer Kathi Roll',
  'price': '750',
  'description': 'Grilled chicken tikka or paneer tikka, green apple relish, mint chutney'},
 {'item_name': 'Pav Bhaji',
  'price': '685',
  'description': 'Classic Bombay street food, spicy mashed potatoes & vegetable mixture,'},
 {'item_name': 'Kheema Ghotala',
  'price': '785',
  'description': 'Bombay street food classic, spiced minced lamb, egg,'},
 {'item_name': 'Goan Pomfret Curry',
  'price': '1325',
  'description': 'Mildly spiced coconut curry flavoured with Garcinia indica rind,'},
 {'item_name': 'Vegetable Club Sandwich',
  'price': '685',
  'description': 'Russian salad, tomato, cheese, iceberg lettuce'},
 {'item_name': 'Vegetarian Panini sandwich',
  'price': '685',
  'description': 'Herbed mushroom, grilled pimentos, avocado, provolone cheese,'},
 {'item_name': 'Classic Club Sa

In [49]:
sample_input = 'get me couple of Classic Club Sandwich along with two Kheema Ghotala'

In [32]:
response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    # messages=[
    #     {"role": "system", "content": "You're a restaurant order extractor. Match spoken order to menu items."},
    #     {"role": "user", "content": f"Menu items: {data}\nCustomer said: {sample_input}\n\nReturn a JSON list with item_name and quantity."}
    # ],
    messages=[
        {
            "role": "system",
            "content": "You're a restaurant order extractor. Match spoken order to menu items and return a JSON list of item_name and quantity."
        },
        {
            "role": "user",
            "content": f"Menu items: {data}\nCustomer said: {sample_input}"
        }
    ],
    temperature=0.2,
    # # functions=[
    #     {
    #         "name": "extract_order",
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "order": {
    #                     "type": "array",
    #                     "items": {
    #                         "type": "object",
    #                         "properties": {
    #                             "item_name": {"type": "string"},
    #                             "quantity": {"type": "integer"}
    #                         },
    #                         "required": ["item_name", "quantity"]
    #                     }
    #                 }
    #             },
    #             "required": ["order"]
    #         }
    #     }
    # ],
    # function_call={"name": "extract_order"}
)

print(response)

ChatCompletion(id='chatcmpl-BM9rYrRmP1hyZCLGTGzb9PaoZajbH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n    {\n        "item_name": "Classic Club Sandwich",\n        "quantity": 2\n    },\n    {\n        "item_name": "Kheema Ghotala",\n        "quantity": 2\n    }\n]\n```', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744620828, model='gpt-4-1106-preview', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=52, prompt_tokens=351, total_tokens=403, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [43]:
function_call = response.choices[0].message.content
# function_call = response.choices[0].message.function_call
# order_data = json.loads(function_call)
# order_data
# Remove markdown code block syntax
json_str = function_call.split('```json\n')[1].split('\n```')[0]
order_items = json.loads(json_str)


In [45]:
order_items

[{'item_name': 'Classic Club Sandwich', 'quantity': 2},
 {'item_name': 'Kheema Ghotala', 'quantity': 2}]

## Trying from free api model GROQ or Langchain

In [46]:
from groq import Groq

client = Groq(api_key=os.getenv('GROQ_API_KEY'))



In [66]:
completion = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "system",
            "content": "You're a restaurant order extractor. Match spoken order to menu items with price and return a just JSON list of item_name, price and price quantity. Nothing other than that"
        },
        {
            "role": "user",
            "content": f"Menu items: {data}\nCustomer said: {sample_input}"
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    # top_p=1,
    # stream=True,
    # stop=None,
)

# for chunk in completion:
#     print(chunk.choices[0].delta.content or "", end="")


In [67]:
output_json = completion.choices[0].message.content
order_items = json.loads(output_json)
print(order_items)

[{'item_name': 'Classic Club Sandwich', 'price': 685, 'quantity': 2}, {'item_name': 'Kheema Ghotala', 'price': 785, 'quantity': 2}]


In [8]:
path = 'models\\m'
import os
if not os.listdir(path):
    print('empty') 
else:
    print('d')

empty


In [ ]:
def download_model(self):
        '''
        
        
        '''
        tokenizer = AutoTokenizer.from_pretrained(self.config['base_model_id'], token=token)
        
        model = AutoModelForCausalLM.from_pretrained(
            self.config['base_model_id'],
            device_map="auto",            # Automatically uses GPU if available
            torch_dtype="auto",           # Adjusts precision
            trust_remote_code=True,
            token= token
        )

        # Optional: wrap in a pipeline
        llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
        
        # Prompt (e.g., for restaurant order understanding)
                
        prompt =[{
                    "role": "system",
                    "content": "You're a restaurant order extractor. Match spoken order to menu items with price and return a just JSON list of item_name, price and price quantity. Nothing other than that"
                },
                {
                    "role": "user",
                    "content": f"Menu items: {data}\nCustomer said: {sample_input}"
                }]
        prompt = json.dumps(prompt)
                
        response = llm(prompt, max_new_tokens=256, do_sample=False)[0]["generated_text"]
        print(response)

### Trying with fine tunned model


In [9]:
sample_input= 'hello may i see the menu'

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
tokenizer = AutoTokenizer.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')

with open('main\components\menu_maneger\extracted_menu_dir\manual_reviewed_menu_data.json') as data:
    data = json.load(data)

prompt = [{
                    "role": "system",
                    "content": "You're a restaurant cashier you have manege the order and well interact with customer"
                },
                {
                    "role": "user",
                    "content": f"Menu items: {data}\nCustomer said: {sample_input}"
                }]



<>:3: SyntaxWarning: invalid escape sequence '\Q'
<>:4: SyntaxWarning: invalid escape sequence '\Q'
<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\Q'
<>:4: SyntaxWarning: invalid escape sequence '\Q'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Admin\AppData\Local\Temp\ipykernel_16068\519800662.py:3: SyntaxWarning: invalid escape sequence '\Q'
  model = AutoModelForCausalLM.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
C:\Users\Admin\AppData\Local\Temp\ipykernel_16068\519800662.py:4: SyntaxWarning: invalid escape sequence '\Q'
  tokenizer = AutoTokenizer.from_pretrained('models\Qwen2.5-0.5B-finetunned-Instruct-merged')
C:\Users\Admin\AppData\Local\Temp\ipykernel_16068\519800662.py:6: SyntaxWarning: invalid escape sequence '\c'
  with open('main\components\menu_maneger\extracted_menu_dir\manual_reviewed_menu_data.json') as data:


In [ ]:
input = tokenizer(prompt, return_tensors='pt')
output = model.generate(**input, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).